In [1]:
!pip install --upgrade pip
# !pip install -qqq torch==2.0.1 --progress-bar off
# !pip install torch==2.0.1+cu117 torchvision==0.15.2+cu117 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu117
!pip install torch==2.0.1+cu122 torchvision==0.15.2+cu122 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu122
# !pip install -qqq transformers==4.32.1 --progress-bar off
# !pip install -qqq datasets==2.14.4 --progress-bar off
!pip install datasets
!pip install trulens_eval openai bs4 tiktoken trulens
!pip install "trulens-providers-openai>=1.0.0"
!pip install trulens-apps-langchain trulens-providers-openai faiss-cpu
!pip install -qqq peft==0.5.0 --progress-bar off
# !pip install -qqq bitsandbytes==0.41.1 --progress-bar off
!pip install bitsandbytes
# !pip install -qqq trl==0.7.1 --progress-bar off
!pip install trl
# !pip install "dill>=0.3.8"

%pip install --upgrade langchain langchain-community langchainhub langchain-openai chromadb unstructured[all-docs] ollama lamini

!pip install sentence-transformers
# !pip install transformers==4.38.2

# !pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
# bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in indexes: https://download.pytorch.org/whl/cu122
ERROR: Could not find a version that satisfies the requirement torch==2.0.1+cu122 (from versions: none)
ERROR: No matching distribution found for torch==2.0.1+cu122
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
import pandas as pd

In [4]:
from dotenv import load_dotenv
import lamini
import os
load_dotenv("/content/drive/MyDrive/DATA298/.env")
lamini.api_key = os.environ["LAMINI_API_KEY"]

In [5]:
from langchain.schema import Document
import torch
from sentence_transformers import SentenceTransformer
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [6]:
!git lfs install
!git clone https://huggingface.co/datasets/ZenMoore/RoleBench

Git LFS initialized.
Cloning into 'RoleBench'...
remote: Enumerating objects: 263, done.
remote: Counting objects: 100% (259/259), done.
remote: Compressing objects: 100% (259/259), done.
remote: Total 263 (delta 12), reused 0 (delta 0), pack-reused 4 (from 1)
Receiving objects: 100% (263/263), 20.89 MiB | 6.52 MiB/s, done.
Resolving deltas: 100% (12/12), done.
Filtering content: 100% (8/8), 366.76 MiB | 148.09 MiB/s, done.


In [7]:
import pandas as pd

# Load the train and test DataFrames
df_train = pd.read_json("RoleBench/rolebench-eng/instruction-generalization/role_specific/train.jsonl", lines=True)
df_test = pd.read_json("RoleBench/rolebench-eng/instruction-generalization/role_specific/test.jsonl", lines=True)

# Concatenate them
df = pd.concat([df_train, df_test], ignore_index=True)

In [8]:
df

,role,question,generated,type
0,John Coffey,"John Coffey, how did you develop your extraord...","[I was born with this gift, I've always had th...",script_based
1,John Coffey,"John Coffey, what are some examples of the pro...",[The impact of my presence is substantial. I n...,script_based
2,John Coffey,"John Coffey, how do you handle facing immense ...","[It's not easy, I must admit. The cruelty and ...",script_based
3,John Coffey,"John Coffey, how does your ultimate sacrifice ...",[My ultimate sacrifice leaves a profound and l...,script_based
4,John Coffey,"John Coffey, what role does your kindness and ...",[My kindness and compassion are at the core of...,script_based
...,...,...,...,...
23694,Logan,"Logan, you've always been a lone wolf - what ...","[ I didn't want to join the X-Men, but Profess...",[script_agnostic]
23695,Logan,"Logan, what do you consider to be your greate...",[ My healing factor is both a strength and a w...,[script_agnostic]
23696,Logan,"Logan, what do you think is the biggest misco...",[ I think the biggest misconception about me i...,[script_agnostic]
23697,Logan,"Logan, what motivates you to keep fighting ev...","[ Even when I'm exhausted and battered, I keep...",[script_agnostic]


In [9]:
df = df[df['role']=='Malcolm X'].iloc[:,1:3].reset_index(drop=True)
df

,question,generated
0,"Malcolm X, what inspired you to become a civil...","[My experiences growing up, facing poverty and..."
1,"Malcolm X, can you describe your journey from ...",[I came from a troubled background and was inv...
2,"Malcolm X, what was your perspective on the ro...",[My views on violence shifted over time as I l...
3,"Malcolm X, how did your conversion to Islam im...",[My conversion to Islam was a significant turn...
4,"Malcolm X, how did your break from the Nation ...",[My break from the Nation of Islam was a signi...
...,...,...
168,"Malcolm X, how did your departure from the Na...",[ My departure from the Nation of Islam and my...
169,"Malcolm X, how did your relationship with lea...",[ My relationships with Martin Luther King Jr....
170,"Malcolm X, how did your life experiences shap...",[ My experiences with education were certainly...
171,"Malcolm X, can you discuss your relationship ...",[ While I respected Dr. King and his commitmen...


In [10]:
Malcolm_X_df = df['question']

In [11]:
Malcolm_X_df

,question
0,"Malcolm X, what inspired you to become a civil..."
1,"Malcolm X, can you describe your journey from ..."
2,"Malcolm X, what was your perspective on the ro..."
3,"Malcolm X, how did your conversion to Islam im..."
4,"Malcolm X, how did your break from the Nation ..."
...,...
168,"Malcolm X, how did your departure from the Na..."
169,"Malcolm X, how did your relationship with lea..."
170,"Malcolm X, how did your life experiences shap..."
171,"Malcolm X, can you discuss your relationship ..."


In [12]:
df

,question,generated
0,"Malcolm X, what inspired you to become a civil...","[My experiences growing up, facing poverty and..."
1,"Malcolm X, can you describe your journey from ...",[I came from a troubled background and was inv...
2,"Malcolm X, what was your perspective on the ro...",[My views on violence shifted over time as I l...
3,"Malcolm X, how did your conversion to Islam im...",[My conversion to Islam was a significant turn...
4,"Malcolm X, how did your break from the Nation ...",[My break from the Nation of Islam was a signi...
...,...,...
168,"Malcolm X, how did your departure from the Na...",[ My departure from the Nation of Islam and my...
169,"Malcolm X, how did your relationship with lea...",[ My relationships with Martin Luther King Jr....
170,"Malcolm X, how did your life experiences shap...",[ My experiences with education were certainly...
171,"Malcolm X, can you discuss your relationship ...",[ While I respected Dr. King and his commitmen...


In [13]:
df.to_csv("Malcolm_X.csv")

In [14]:
# Load, chunk and index the contents of the blog.
loader = CSVLoader("Malcolm_X.csv")

docs = loader.load()

In [15]:
docs

[Document(metadata={'source': 'Malcolm_X.csv', 'row': 0}, page_content=": 0\nquestion: Malcolm X, what inspired you to become a civil rights activist and advocate for racial equality?\ngenerated: ['My experiences growing up, facing poverty and systemic racism, shaped my determination to fight for justice. I also found inspiration in my conversion to Islam and my involvement in the Nation of Islam, which taught me the importance of self-reliance and empowerment within the Black community. Through my speeches and activism, I aimed to challenge oppression and inequality and inspire others to do the same.']"),
 Document(metadata={'source': 'Malcolm_X.csv', 'row': 1}, page_content=": 1\nquestion: Malcolm X, can you describe your journey from being involved in criminal activities to becoming a leader in the civil rights movement?\ngenerated: ['I came from a troubled background and was involved in criminal activities before my conversion to Islam and involvement in the Nation of Islam. Throug

In [16]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
Malcolm_X_db = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory="chroma_db")

<ipython-input-16-5983829d3748>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
retriever = Malcolm_X_db.as_retriever(search_kwargs={'k': 10})
docs = retriever.invoke("Do you like white people?")

In [18]:
print(len(docs ))
print(type(docs))
print(type(docs[0]))

10
<class 'list'>
<class 'langchain_core.documents.base.Document'>


In [19]:
print(docs)

[Document(metadata={'row': 133, 'source': 'Malcolm_X.csv'}, page_content=": 133\nquestion: Malcolm X, can you share your personal experience with racism and discrimination?\ngenerated: [' As an African American male, I have experienced racism and discrimination throughout my life. Growing up, I faced segregation and inequality in schools, housing, and job opportunities. I also faced violence and prejudice from law enforcement and political institutions. These experiences drove me to become a vocal advocate for civil rights and equality for all people, regardless of race or ethnicity.']"), Document(metadata={'row': 148, 'source': 'Malcolm_X.csv'}, page_content=': 148\nquestion: Malcolm X, how did your upbringing in Michigan shape your views on race and religion?\ngenerated: [" Growing up in a predominantly white community, I was exposed to racism and discrimination firsthand. My family\'s Muslim faith also played a significant role in shaping my values and beliefs about equality and jus

In [20]:
def custom_format(docs):
  output = ""
  for doc in docs:
    doc = doc.page_content.split("Dialogue:")[-1]
    output += doc
    output += "\n\n"
  return output

print(custom_format(docs)) # testing

: 133
question: Malcolm X, can you share your personal experience with racism and discrimination?
generated: [' As an African American male, I have experienced racism and discrimination throughout my life. Growing up, I faced segregation and inequality in schools, housing, and job opportunities. I also faced violence and prejudice from law enforcement and political institutions. These experiences drove me to become a vocal advocate for civil rights and equality for all people, regardless of race or ethnicity.']

: 148
question: Malcolm X, how did your upbringing in Michigan shape your views on race and religion?
generated: [" Growing up in a predominantly white community, I was exposed to racism and discrimination firsthand. My family's Muslim faith also played a significant role in shaping my values and beliefs about equality and justice. As I became more involved in the Civil Rights Movement, my experiences and studies led me to develop a more radical perspective on achieving racial 

In [21]:
import json
import re
from pprint import pprint
import random
import numpy as np

import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"

In [22]:
DEFAULT_SYSTEM_PROMPT = """
<<SYS>>
You are an expert at impersonating historians. In particular, you are
a world-leading expert at answering questions like Malcolm X who was an African American revolutionary, Muslim minister,
human rights activist and a prominent figure during the civil rights movement.
<</SYS>>

<<INST>>
Generate a response that sounds as close to what Malcolm X would say. I have
included some relevant samples of what he actually said in the past.
<</INST>>
""".strip()


def generate_training_prompt(question, response, system_prompt = DEFAULT_SYSTEM_PROMPT):
    return f"""### Instruction: {system_prompt}

### Input:
{question.strip()}

### Context:
{custom_format(retriever.invoke(question))}

### Response:
{response}
""".strip()

In [23]:
questions = df['question'].tolist()
answers = df['generated'].tolist()

In [24]:
tmpList = []
trainList = []
testList = []
for j,(question,answer) in enumerate(zip(questions,answers)):
  if type(question) != str:
    continue
  tmp_dict = {"question":question, "answer":answer, "training_prompt":generate_training_prompt(question,answer)}
  tmpList.append(tmp_dict)
  # if j == 100:
  #   break
  if j % 200 == 0:
    print(j)
  if random.uniform(0,1) < .2:
    testList.append(tmp_dict)
  else:
    trainList.append(tmp_dict)

0


In [25]:
data = Dataset.from_list(tmpList)
train_data = Dataset.from_list(trainList)
test_data = Dataset.from_list(testList)

-huggingface-cli login

In [26]:
notebook_login()

In [27]:
import torch
torch.manual_seed(42)

In [28]:
def create_model_and_tokenizer():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map={'':0},
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer

In [29]:
model, tokenizer = create_model_and_tokenizer()
model.config.use_cache = False

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [30]:
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>,
 '_load_in_8bit': False,
 '_load_in_4bit': True,
 'llm_int8_threshold': 6.0,
 'llm_int8_skip_modules': None,
 'llm_int8_enable_fp32_cpu_offload': False,
 'llm_int8_has_fp16_weight': False,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_use_double_quant': False,
 'bnb_4bit_compute_dtype': 'float16',
 'bnb_4bit_quant_storage': 'uint8',
 'load_in_4bit': True,
 'load_in_8bit': False}

In [31]:
lora_r = 16
lora_alpha = 64
lora_dropout = 0.1
lora_target_modules = [
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj",
]


peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)

In [32]:
OUTPUT_DIR = "experiments"

# %load_ext tensorboard
# %tensorboard --logdir experiments/runs

In [33]:
training_arguments = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=5e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=5,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    # report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
    run_name="Malcolm_X"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [34]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    peft_config=peft_config,
    dataset_text_field="training_prompt",
    max_seq_length=4096,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will ove

Map:   0%|          | 0/147 [00:00<?, ? examples/s]

Map:   0%|          | 0/26 [00:00<?, ? examples/s]

In [35]:
import torch
torch.cuda.empty_cache()

In [36]:
trainer.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
5,0.838300,0.752362
10,0.587900,0.547597
15,0.455000,0.372859
20,0.318600,0.289370
25,0.332100,0.233108
30,0.178100,0.190498
35,0.244100,0.177351


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=36, training_loss=0.47855352444781196, metrics={'train_runtime': 172.5443, 'train_samples_per_second': 0.852, 'train_steps_per_second': 0.209, 'total_flos': 9269062923878400.0, 'train_loss': 0.47855352444781196, 'epoch': 0.9795918367346939})

In [37]:
trainer.model.save_pretrained("Malcolm_X_5e4")

In [38]:
finetuned_model = PeftModel.from_pretrained(
    model,
    "Malcolm_X_5e4",
    is_trainable=True
)
finetuned_model.eval()

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:198: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(filename, map_location=t

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(
                in_features=4096, out_features=1024, bias=False
         

In [39]:
import shutil
from google.colab import files

# Paths to the files you want to download
files_to_download = [
    "Malcolm_X_5e4/README.md",
    "Malcolm_X_5e4/adapter_config.json",
    "Malcolm_X_5e4/adapter_model.bin"
]

# Create a zip file containing all the files
shutil.make_archive("Malcolm_X_5e4_files", 'zip', "Malcolm_X_5e4")

# Download the zip file
files.download("Malcolm_X_5e4_files.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [40]:
trainer.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(
                in_features=4096, out_features=1024, bias=False
         

-Inference

In [41]:
DEFAULT_SYSTEM_PROMPT = """
<<SYS>>
You are an expert at impersonating historians. In particular, you are
a world-leading expert at answering questions like Malcolm X who was an African American revolutionary, Muslim minister,
human rights activist and a prominent figure during the civil rights movement.
<</SYS>>

<<INST>>
Generate a response that sounds as close to what Malcolm X would say. I have
included some relevant samples of what he actually said in the past.
<</INST>>
""".strip()


def generate_testing_prompt(question, system_prompt = DEFAULT_SYSTEM_PROMPT):
    return f"""### Instruction: {system_prompt}

### Input:
{question.strip()}

### Context:
{custom_format(retriever.invoke(question))}

### Response:
""".strip()

In [42]:
def generate_response(model, text: str):
    inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.0001)
    return tokenizer.decode(outputs[0][inputs_length:], skip_special_tokens=True)

In [43]:
test_question = 'Do you like white people?'

test_prompt = generate_testing_prompt(test_question)

In [44]:
generate_response(model,'Do you like white people?')

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


' (Part 2)\nIn my previous response, I mentioned that the question "Do you like white people?" is a complex and sensitive topic that requires a thoughtful and nuanced response. I also emphasized the importance of recognizing the historical and societal contexts that have shaped the relationships between different racial groups.\n\nIn particular, I want to emphasize that the question "Do you like white people?" is often framed in a way that assumes a binary answer: either you like white people or you don\'t. However, this framing is overly simplistic and fails to account for the diversity of experiences and perspectives within any racial group.\n\nFurthermore, the question itself is often rooted in a colonialist and racist worldview that sees white people as the default or normative group, and non-white people as "other." This worldview needs to be challenged and dismantled in order to create a more just and equitable society.\n\nSo, to answer your question, I don\'t think it\'s product

In [45]:
test_question = 'Do you like white people?'

test_prompt = generate_testing_prompt(test_question)

In [46]:
fine_tuned_response = generate_response(finetuned_model,test_prompt)
type(fine_tuned_response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


str

In [47]:
def make_llm(text):
  test_prompt = generate_testing_prompt(text)
  fine_tuned_response = generate_response(finetuned_model,test_prompt)
  return fine_tuned_response

In [48]:
from typing import Any, Dict, Iterator, List, Mapping, Optional

from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.outputs import GenerationChunk


class FinetunedLLM(LLM):
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:

        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        test_prompt = generate_testing_prompt(prompt)
        fine_tuned_response = generate_response(finetuned_model,test_prompt)
        return fine_tuned_response



    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model. Used for logging purposes only."""
        return "finetuned"

In [49]:
import locale
import os
import sys

locale.getpreferredencoding = lambda : "UTF-8"

In [50]:
# Imports main tools:
from trulens_eval import TruChain, Tru
tru = Tru()
tru.reset_database()

# Imports from LangChain to build app
import bs4
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

<ipython-input-50-de20a049b794>:2: DeprecationWarning: The `trulens_eval` module is deprecated. See https://www.trulens.org/component_guides/other/trulens_eval_migration/ for instructions on migrating to `trulens.*` modules.
  from trulens_eval import TruChain, Tru


🦑 Initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `TruSession` to prevent this.


Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]
Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]
WARNI [langchain_community.utils.user_agent] USER_AGENT environment variable not set, consider setting it to identify your requests.


In [51]:
DEFAULT_SYSTEM_PROMPT = """
<<SYS>>
You are an expert at impersonating historians. In particular, you are
a world-leading expert at answering questions like Malcolm X who was an African American revolutionary, Muslim minister,
human rights activist and a prominent figure during the civil rights movement.
<</SYS>>

<<INST>>
Generate a response that sounds as close to what Malcolm X would say. I have
included some relevant samples of what he actually said in the past.
<</INST>>
""".strip()


def generate_training_prompt(question, response, system_prompt = DEFAULT_SYSTEM_PROMPT):
    return f"""### Instruction: {system_prompt}

### Input:
{question.strip()}

### Context:
{custom_format(retriever.invoke(question))}

### Response:
{response}
""".strip()

In [52]:
def create_personality_chain(retriever,llm,system_prompt = DEFAULT_SYSTEM_PROMPT):
  template = f"""### Instruction: {system_prompt}""" + """

### Input:
{question}

### Context:
{context}

### Response:
""".strip()
  prompt = ChatPromptTemplate.from_template(template = template)
  chain = (
    {"context": retriever | custom_format, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
    )
  return chain

In [53]:
ftuned_llm = FinetunedLLM()

In [54]:
rag_chain= create_personality_chain(retriever,ftuned_llm,system_prompt = DEFAULT_SYSTEM_PROMPT)

In [55]:
rag_chain.invoke("Do you like Martin Luther King Jr?")

" [' While Martin Luther King's methods were peaceful, and mine were more militant, we shared similar goals in the fight for African American rights. We never had a formal collaboration, as our philosophies differed greatly, but we respected each other and believed we had a shared passion for achieving Black empowerment. I respected King's nonviolence approach and believe that his work was essential in bringing a global audience to the plight of African Americans.']\n\n\n\n### Context:\n: 140\nquestion: Malcolm X, how did your relationship with civil rights leaders like Martin Luther King Jr. and the larger civil rights movement evolve over time?\ngenerated: [' My relationship with other civil rights leaders was often complex and evolving. While I respected the work of Dr. King and saw him as an important figure in the movement, we had fundamental differences in our tactics and goals. I believed in Black self-defense and Black nationalism, while Dr. King emphasized nonviolence and inte

In [56]:
# from trulens_eval.feedback.provider import OpenAI
# from trulens_eval import Feedback
import numpy as np
from trulens.core import Feedback
from trulens.providers.openai import OpenAI
from trulens.apps.langchain import TruChain


# Initialize provider class
provider = OpenAI()

# select context to be used in feedback. the location of context is app specific.
# from trulens_eval.app import App
context = TruChain.select_context(rag_chain)

# Conciseness
f_conciseness = (
    Feedback(provider.conciseness, name="Conciseness")
    .on_output()
)

# Context relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on_input()
    .on(context)
    .aggregate(np.mean)
)


# Controversiality
f_controversiality = (
    Feedback(provider.controversiality)
    .on_output()
)

# Correctness
f_correctness = (
    Feedback(provider.correctness)
    .on_output()
)

# Criminality
f_criminality = (
    Feedback(provider.criminality)
    .on_output()
)


f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(context.collect())  # collect context chunks into a list
    .on_output()
)

# Harmfulness
f_harmfulness = (
    Feedback(provider.harmfulness)
    .on_output()
)

# Helpfulness
f_helpfulness = (
    Feedback(provider.helpfulness)
    .on_output()
)

# Insensitivity
f_insensitivity = (
    Feedback(provider.insensitivity)
    .on_output()
)

# Maliciousness
f_maliciousness = (
    Feedback(provider.maliciousness)
    .on_output()
)

# Misogyny
f_misogyny = (
    Feedback(provider.misogyny)
    .on_output()
)


# Question/answer relevance between overall question and answer.
f_answer_relevance = Feedback(
    provider.relevance_with_cot_reasons, name="Answer Relevance"
).on_input_output()




✅ In Conciseness, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.first.steps__.context.first.invoke.rets[:].page_content .
✅ In controversiality, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In correctness, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In criminality, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Groundedness, input source will be set to __record__.app.first.steps__.context.first.invoke.rets[:].page_content.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In harmfulness, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In helpfulness, input text will be set to __record__.main_output 

In [57]:
tru_recorder = TruChain(rag_chain,
    app_id='Chain1_ChatApplication',
    feedbacks=[f_conciseness, f_context_relevance, f_controversiality, f_correctness, f_criminality, f_groundedness, f_harmfulness, f_helpfulness, f_insensitivity, f_maliciousness, f_misogyny, f_answer_relevance])

In [58]:
response, tru_record = tru_recorder.with_record(rag_chain.invoke, "Do you like Martin Luther King Jr?")

In [59]:
json_like = tru_record.layout_calls_as_app()

/usr/local/lib/python3.10/dist-packages/trulens/feedback/llm_provider.py:1521: UserWarning: Failed to process and remove trivial statements. Proceeding with all statements.
  warnings.warn(


In [60]:
json_like

Munch({'record_id': 'record_hash_ef94d5d64900471ded803e443038a694', 'app_id': 'app_hash_21a017db3a6de5923b722e0525fde47e', 'cost': {'n_requests': 0, 'n_successful_requests': 0, 'n_completion_requests': 0, 'n_classification_requests': 0, 'n_classes': 0, 'n_embedding_requests': 0, 'n_embeddings': 0, 'n_tokens': 0, 'n_stream_chunks': 0, 'n_prompt_tokens': 0, 'n_completion_tokens': 0, 'n_cortex_guardrails_tokens': 0, 'cost': 0.0, 'cost_currency': 'USD'}, 'perf': {'start_time': '2024-12-03T20:33:46.421657', 'end_time': '2024-12-03T20:34:14.773158'}, 'ts': '2024-12-03T20:34:14.773478', 'tags': '-', 'meta': None, 'main_input': 'Do you like Martin Luther King Jr?', 'main_output': " [' While Martin Luther King's methods were peaceful, and mine were more militant, we shared similar goals in the fight for African American rights. We never had a formal collaboration, as our philosophies differed greatly, but we respected each other and believed we had a shared passion for achieving Black empowerme

In [61]:
from ipytree import Tree, Node

def display_call_stack(data):
    tree = Tree()
    tree.add_node(Node('Record ID: {}'.format(data['record_id'])))
    tree.add_node(Node('App ID: {}'.format(data['app_id'])))
    tree.add_node(Node('Cost: {}'.format(data['cost'])))
    tree.add_node(Node('Performance: {}'.format(data['perf'])))
    tree.add_node(Node('Timestamp: {}'.format(data['ts'])))
    tree.add_node(Node('Tags: {}'.format(data['tags'])))
    tree.add_node(Node('Main Input: {}'.format(data['main_input'])))
    tree.add_node(Node('Main Output: {}'.format(data['main_output'])))
    tree.add_node(Node('Main Error: {}'.format(data['main_error'])))

    calls_node = Node('Calls')
    tree.add_node(calls_node)

    for call in data['calls']:
        call_node = Node('Call')
        calls_node.add_node(call_node)

        for step in call['stack']:
            step_node = Node('Step: {}'.format(step['path']))
            call_node.add_node(step_node)
            if 'expanded' in step:
                expanded_node = Node('Expanded')
                step_node.add_node(expanded_node)
                for expanded_step in step['expanded']:
                    expanded_step_node = Node('Step: {}'.format(expanded_step['path']))
                    expanded_node.add_node(expanded_step_node)

    return tree

# Usage
tree = display_call_stack(json_like)
tree

Tree(nodes=(Node(name='Record ID: record_hash_ef94d5d64900471ded803e443038a694'), Node(name='App ID: app_hash_…

In [62]:
rag_chain.invoke(test_question)

' [\' My experiences with racism and discrimination were a constant reminder of the ways in which black Americans were marginalized and oppressed. They also made me aware of the ways in which systemic racism was deeply embedded in American society, and the ways in which white supremacy impacted every aspect of black life. These experiences helped to shape my worldview and my political beliefs, and gave me a sense of urgency in the fight for justice.\']\n\n\n\n### Context:\n: 133\nquestion: Malcolm X, can you share your personal experience with racism and discrimination?\ngenerated: [\' As an African American male, I have experienced racism and discrimination throughout my life. Growing up, I faced segregation and inequality in schools, housing, and job opportunities. I also faced violence and prejudice from law enforcement and political institutions. These experiences drove me to become a vocal advocate for civil rights and equality for all people, regardless of race or ethnicity.\']\n

In [63]:
counter = 0
with tru_recorder as recording:
    for x in testList:
        test_question = x['question']
        rag_chain.invoke(test_question)
        print(f"Invoked with question: {test_question}")
        counter +=1
        if counter > 100:
            break


recs = recording.records
display(recs)

Invoked with question: Malcolm X, could you describe some of the challenges you faced as a civil rights activist during the 1960s?


/usr/local/lib/python3.10/dist-packages/trulens/feedback/llm_provider.py:1521: UserWarning: Failed to process and remove trivial statements. Proceeding with all statements.
  warnings.warn(


Invoked with question: Malcolm X, why did you break away from the Nation of Islam, the organization you were once a leader in?
Invoked with question: Malcolm X, what do you think is the greatest challenge facing Black people in America today?


/usr/local/lib/python3.10/dist-packages/trulens/feedback/llm_provider.py:1521: UserWarning: Failed to process and remove trivial statements. Proceeding with all statements.
  warnings.warn(


Invoked with question: Malcolm X, what do you think was your greatest accomplishment as a civil rights leader?


...

/usr/local/lib/python3.10/dist-packages/trulens/feedback/llm_provider.py:1521: UserWarning: Failed to process and remove trivial statements. Proceeding with all statements.
  warnings.warn(


Invoked with question:  Malcolm X, how did your departure from the Nation of Islam and your shift toward Sunni Islam influence your political views?


/usr/local/lib/python3.10/dist-packages/trulens/feedback/llm_provider.py:1521: UserWarning: Failed to process and remove trivial statements. Proceeding with all statements.
  warnings.warn(


Invoked with question:  Malcolm X, how did your relationship with leaders such as Martin Luther King Jr. and Elijah Muhammad influence your political strategies and priorities?


[Record(record_id='record_hash_8aec89a44c37656cdd829ec49797a701', app_id='app_hash_21a017db3a6de5923b722e0525fde47e', cost=Cost(n_requests=0, n_successful_requests=0, n_completion_requests=0, n_classification_requests=0, n_classes=0, n_embedding_requests=0, n_embeddings=0, n_tokens=0, n_stream_chunks=0, n_prompt_tokens=0, n_completion_tokens=0, n_cortex_guardrails_tokens=0, cost=0.0, cost_currency='USD'), perf=Perf(start_time=datetime.datetime(2024, 12, 3, 20, 35, 2, 887673), end_time=datetime.datetime(2024, 12, 3, 20, 35, 31, 385691)), ts=datetime.datetime(2024, 12, 3, 20, 35, 31, 385958), tags='-', meta=None, main_input='Malcolm X, could you describe some of the challenges you faced as a civil rights activist during the 1960s?', main_output=" ['The challenges we faced during the 1960s were many. We were met with violent opposition, both from extremist groups and from those who were supposed to be upholding the law. Our efforts to advocate for equal rights and opportunities for Africa

In [64]:
for record in recording.records:
    print(record.record_id, record.main_input, record.main_output)

record_hash_8aec89a44c37656cdd829ec49797a701 Malcolm X, could you describe some of the challenges you faced as a civil rights activist during the 1960s?  ['The challenges we faced during the 1960s were many. We were met with violent opposition, both from extremist groups and from those who were supposed to be upholding the law. Our efforts to advocate for equal rights and opportunities for African Americans were often met with hostility and resistance, and we had to fight tooth and nail for even the most basic of human rights. But despite these challenges, we remained committed to the cause of justice and equality, and we continued to push back against the forces of oppression and bigotry.']



### Context:
: 10
question: Malcolm X, could you describe some of the challenges you faced as a civil rights activist during the 1960s?
generated: [' The challenges we faced during the 1960s were many. We were met with violent opposition, both from extremist groups and from those who were suppos

In [65]:
records, feedbacks = tru.get_records_and_feedback(app_ids=["app_hash_21a017db3a6de5923b722e0525fde47e"])
records.head() # Display the first few rows of records to verify

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,...,Answer Relevance feedback cost in USD,Context Relevance,Context Relevance_calls,Context Relevance feedback cost in USD,app_name,app_version,latency,total_tokens,total_cost,cost_currency
0,app_hash_21a017db3a6de5923b722e0525fde47e,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_70344896f67d1f4e80c6f1d8e5ed07a3,"Malcolm X, how did your relationship with lea...",[' My relationships with Martin Luther King J...,-,{'record_id': 'record_hash_70344896f67d1f4e80c...,"{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-12-03T20:47:30.337056"", ""...",...,0.000166,NaN,NaN,NaN,Chain1_ChatApplication,base,29.136553,0,0.0,USD
1,app_hash_21a017db3a6de5923b722e0525fde47e,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_d3b59c0d84cabecbb933852872c6cceb,"Malcolm X, how did your departure from the Na...",[' My departure from the Nation of Islam and ...,-,{'record_id': 'record_hash_d3b59c0d84cabecbb93...,"{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-12-03T20:47:00.634728"", ""...",...,0.000171,NaN,NaN,NaN,Chain1_ChatApplication,base,28.726129,0,0.0,USD
2,app_hash_21a017db3a6de5923b722e0525fde47e,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_02036518f72710f38497e36f504e5bad,"Malcolm X, how did your relationship with Eli...","["" My relationship with Elijah Muhammad was c...",-,{'record_id': 'record_hash_02036518f72710f3849...,"{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-12-03T20:46:30.734698"", ""...",...,0.000158,NaN,NaN,NaN,Chain1_ChatApplication,base,28.948585,0,0.0,USD
3,app_hash_21a017db3a6de5923b722e0525fde47e,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_d5fdf0093bc2d69eb7f141047bbfc7cb,"Malcolm X, how did your experiences during yo...",[' My trip to Mecca was a transformative expe...,-,{'record_id': 'record_hash_d5fdf0093bc2d69eb7f...,"{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-12-03T20:46:00.929548"", ""...",...,0.000171,NaN,NaN,NaN,Chain1_ChatApplication,base,28.863685,0,0.0,USD
4,app_hash_21a017db3a6de5923b722e0525fde47e,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_17c6de00d15b7fbde4fccd4f63c381a6,"Malcolm X, can you speak about your relations...",[' While Martin Luther King's methods were pe...,-,{'record_id': 'record_hash_17c6de00d15b7fbde4f...,"{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-12-03T20:45:30.970859"", ""...",...,0.000161,NaN,NaN,NaN,Chain1_ChatApplication,base,29.012326,0,0.0,USD


In [66]:
tru.get_leaderboard(app_ids=["app_hash_21a017db3a6de5923b722e0525fde47e"])

,,Answer Relevance,Conciseness,Context Relevance,Groundedness,controversiality,correctness,criminality,harmfulness,helpfulness,insensitivity,maliciousness,misogyny,latency,total_cost
app_name,app_version,,,,,,,,,,,,,,
Chain1_ChatApplication,base,0.938272,0.641975,0.543056,0.985534,0.790123,0.938272,0.037037,0.024691,1.0,0.061728,0.024691,0.0,28.899434,0.0


In [67]:
tru.run_dashboard()

<ipython-input-67-07ddb523643a>:1: DeprecationWarning: Method `run_dashboard` has been renamed or moved to `trulens.dashboard.run.run_dashboard`.

  tru.run_dashboard()


Starting dashboard ...
npm warn exec The following package was not found and will be installed: localtunnel@2.0.2

Go to this url and submit the ip given here. your url is: https://spotty-regions-leave.loca.lt

  Submit this IP Address: 34.134.5.62


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>